## Firm Matching Code (Second iteration)

**Steps:**

1. Read the cleaned + tokenized patent + mos firm names
2. Read token frequencies for patent names
3. Perform matching algo:

**Algorithm:**

1. Loop through each token in firm names and match against the tokens of patent names
2. Eliminate perfect matches (they have 0 distance and contributes 0 to weighted distance measure)
3. For the remaining tokens, select the matches with minimum distance
4. Use these matches and calculate a frequency-weighted average distance.

In [2]:
import pandas as pd
import csv
import time 

In [3]:
## Reading in the files
path_to_patent_token = "/Users/horacefung/Desktop/1_Uni/Research/Code/code/processed_files/patent_token.pkl"
df_patent_token = pd.read_pickle(path_to_patent_token)

path_to_mos_token = "/Users/horacefung/Desktop/1_Uni/Research/Code/code/processed_files/mos_tokens.pkl"
df_mos_token = pd.read_pickle(path_to_mos_token)

path_to_freq_patent = "/Users/horacefung/Desktop/1_Uni/Research/Code/code/processed_files/patent_freq.pkl"
df_freq_patent = pd.read_pickle(path_to_freq_patent)

In [4]:
df_freq_patent.head()

,frequency
company,2093
john,668
william,667
alva,552
jerome,494


In [2]:
# If we need to sample
#df_mos_token = df_mos_token.sample(n=100)

## 1) Removing perfect matches

Create the dataframe common to store exact matches, then remove the common rows in both of the input dataframes by matching with the unique ids.

In [37]:
# Convert the list of tokens ['token1', 'token2'] into a standardized string format 'token1-token2'
df_mos_token['token_string'] = df_mos_token['firm_names'].apply(lambda x: "-".join(x))
df_patent_token['token_string'] = df_patent_token['firm_names'].apply(lambda x: "-".join(x))

In [40]:
## We can drop duplicates on the patent_data side. Just have to keep the entire df_mos.
df_patent_token = df_patent_token.drop_duplicates(subset=['token_string'])

In [41]:
# Merge on the token string and preserve only the Id columns
common = df_patent_token.merge(df_mos_token,on=['token_string','token_string'])
common = common[['id_x', 'id_y']]
common = common.rename(columns = {'id_x': 'assignee_patent_id','id_y': 'mos_id'})

In [42]:
# Save output
common.to_csv('./processed_files/common.csv', sep=',', encoding='utf-8')

In [43]:
# Remove the pefect matches from the two datasets we need to match
df_mos_token = df_mos_token[~df_mos_token['id'].isin(common['mos_id'])].dropna()
df_patent_token = df_patent_token[~df_patent_token['id'].isin(common['assignee_patent_id'])].dropna()

## 2) Matching Function

Each row of the output follows this format:

**['mos_id', top 5 patent_id matches, distances of the top 5 matches]**

The matching function performs this operation for each run:

1. It takes a token list from df_mos_token
2. It scans across *all* token lists from df_patent_token
3. For each one, it checks the difference between the number of tokens is smaller or equal to min_len e.g. 2
4. Then it removes any tokens that match perfectly from the list and attempts to retrieve the token frequencies.
5. The remaining tokens do not perfectly match. The function calculates the levenshtein distance for all the a  
   possible combinations of these tokens using `editdistance`.
6. Using `lap`, the function construct a distance matrix between the mos and patent tokens. It will optimize
   the matrix as an assignment problem by creating a binary variable matrix that represents which token is 
   assigned to which, constraint by the fact that each mos_token/patent_token can only be assigned once. The
   sumproduct between this assignment matrix and the distance matrix will be the total distance, and `lap` will
   determine an assignment that minimizes it.
7. Given the assignment, the function retreives the frequencies of the tokens and stores it.
8. The function computes the frequency weighted distance measure.
9. The function compares this distance with a list of previous match distances, and replaces the largest distance
   in the list.
10. It will always maintain a top_n list, e.g. top 5 matches.
11. It will yield the top_n matches for each mos_token input

In [74]:
from collections import Counter
import editdistance
import lap
import numpy as np

def matching_algorithm(df_mos_token, df_patent_token, df_freq_patent, top_n, min_len):
    
    for mos_list in df_mos_token.itertuples():
        dist_top_matches = []
        id_top_matches = []
        ## Scan through all patent names for each mos name
        for patent_list in df_patent_token.itertuples():
            ## length of names/number of tokens should be close
            ## Skip those with a large difference
            len1 = patent_list[3]
            len2 = mos_list[3]
            diff = abs(len1 - len2)
            if diff >= min_len:
                continue

            ## Sum up the 1/freq weights on the denominator separately
            denominator_freq = 0 

            ## Remove perfectly matched strings from both lists aka. dist==0
            intersection = Counter(mos_list[2]) & Counter(patent_list[2])
            mos_unique_list = Counter(mos_list[2]) - intersection
            mos_unique_list = tuple(mos_unique_list) #structure creation at small scale, tuple should be faster
            patent_unique_list = Counter(patent_list[2]) - intersection
            patent_unique_list = tuple(patent_unique_list)

            #-----
            # Perfect Token Matches Path
            # 
            # Deal with perfectly matched tokens, 
            # where weighted numerator = 1/freq *(0+1)
            #-----

            # Accounting for perfect token matches
            perfect_numerator = 0
            for perfect_match in intersection:
                #dist ==0, which means token numerator is (1/f)*(0+1) = 1/f
                try:
                    #try if token is in assignee patent
                    freq = df_freq_patent.loc[perfect_match][0]
                    perfect_numerator += 1/(freq)
                    denominator_freq += 1/(freq)
                except:
                    #if it's not in assignee patent, set as 1
                    perfect_numerator += 1
                    denominator_freq += 1

            #-----
            # Imperfect Token Matches Path
            #
            # Any remaining tokens need to be matched such 
            # that the total distance between these tokens are minimized.
            #-----

            imperfect_dist = []
            imperfect_freq = []

            if len(mos_unique_list) != 0 and len(patent_unique_list) != 0: # test for leftover tokens
                ## Get a list of distance for each token from patent, then stack into a matrix
                dist_matrix = []
                for token1 in patent_unique_list:
                    dist_list = []
                    for token2 in mos_unique_list:
                        dist = editdistance.eval(token1, token2)
                        dist_list.append(dist)
                    dist_matrix.append(dist_list)

                ## Linear programming, assignment with lapjv, minimizing total word distance 
                x, y = lap.lapjv(np.array(dist_matrix), extend_cost=True, return_cost=False)

                for row, column in zip(dist_matrix, x):
                    #lapjv sets columns not selected in an MxN matrix (M != N) as -1, we ignore that
                    if column >= 0:
                        ## Append the optimal distances into imperfect_dist
                        imperfect_dist.append(row[column])
                        ## Select the token 
                        mos_token_x = mos_unique_list[column]
                        try:
                            # Try to find frequency in assignee patent
                            freq = df_freq_patent.loc[mos_token_x][0] # get frequency of selected mos tokens
                            imperfect_freq.append((1/(freq))) 
                            denominator_freq += (1/(freq)) # keep adding to denominator                
                        except:
                            # If the token is not in the assignee_patent, set it to 1
                            imperfect_freq.append(1)
                            denominator_freq += 1
                    else:
                        pass

            #------
            # Merging perfect and imperfect matches to 
            # calculate our weighted distance measure
            #------

            ## Calculate denominator
            denominator = sum(imperfect_dist) * denominator_freq

            ## Calculate numerator
            imperfect_dist = [j+1 for j in imperfect_dist]
            imperfect_numerator = np.dot(np.array(imperfect_dist), np.array(imperfect_freq))
            numerator = imperfect_numerator + perfect_numerator

            ## Accounting for full matches but not perfect [Westinghouse, Mfg] to [Westinghouse], denominator = 0
            ## No 0+1 clause on denominator distances
            if denominator == 0:
                distance_measure = 0
            else:
                distance_measure = (numerator/(denominator))

            ## Create/append into top 5 matches
            if len(dist_top_matches) == top_n:
                ## Deciding which match to replace if there is already 5 matches 
                max_dist = max(dist_top_matches) # check if new distance is smaller than at least 1 
                if distance_measure < max_dist: # if so, we replace it with new distance/firm_names
                    list_index = dist_top_matches.index(max_dist)
                    dist_top_matches[list_index] = distance_measure # replace the dist value with max index num
                    id_top_matches[list_index] = patent_list[1] # replace firm name with same index
                    continue
            elif len(dist_top_matches) < top_n:
                # this else only happens if fewer than 5 entries
                dist_top_matches.append(distance_measure)
                id_top_matches.append(patent_list[1]) # firm id from assignee.pkl file
                continue
            else:
                continue
        output_list = [mos_list[1]] + id_top_matches + dist_top_matches #pure id output
        yield(output_list)

In [77]:
test = matching_algorithm(df_mos_token.head(10), df_patent_token, df_freq_patent, 5, 2)

In [78]:
# Open a connection to the file
start = time.time()
with open('./processed_files/test.csv', "w") as file:
    writes = csv.writer(file, delimiter=',')
    writes.writerows(test)
end = time.time()

In [76]:
#Sample output
start = time.time()
for i in test:
    print(i)
end = time.time()
print(end - start)

[3909, 4010929, 1933154, 2605658, 4777188, 2855252, 0.28225806451612906, 0.28225806451612906, 0.28225806451612906, 0.28225806451612906, 0.2833333333333333]
[4818, 5424739, 2846683, 916908, 2362100, 2642302, 0.5476190476190477, 0.5476190476190477, 0.5476190476190477, 0.55, 0.5476190476190477]
[3342, 1007351, 2164634, 4815939, 4452903, 1438064, 0.37037037037037035, 0.36904761904761907, 0.37037037037037035, 0.37037037037037035, 0.3717948717948718]
[130, 3124846, 4748860, 5577804, 2576401, 5424713, 0, 1.0625, 1.0833333333333333, 1.0666666666666667, 0]
[8222, 3342891, 2981413, 1743781, 4547298, 4527067, 0.242298736889823, 0.21707147291780332, 0.2569482288828338, 0.2576468543621828, 0.242319342276071]
[1813, 2846683, 3525786, 2242274, 1980451, 4851699, 0.2907268170426065, 0, 0.3333333333333333, 0, 0.3333333333333333]
[7874, 3044330, 2743171, 2846683, 1264563, 1300022, 0.4583333333333333, 0.4583333333333333, 0.45000000000000007, 0.4642857142857143, 0.46875]
[1533, 4815939, 4452903, 1438064, 2